In [318]:
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.pipeline import make_pipeline
from yellowbrick.regressor import residuals_plot

from notebooks.Dzim.data_mining.validation.training import estimate_multiple_models, estimate_model
from src.tools.predictors import plot_prediction
from src.tools.predictors import plot_timeseries

In [2]:
plt.rcParams['figure.facecolor'] = 'white'

In [3]:
%load_ext autoreload
%autoreload 2

#### Loading data

In [34]:
data = pd.read_csv('data1.csv', index_col=0)

In [342]:
data.head(100)

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
...,...,...
96,0,@user i'll always hope that one day i'll get t...
97,0,#model i love u take with u all the time in ...
98,0,couple having sex fat naked japanese girls


In [ ]:
data_sample = pd.DataFrame({
    'tweet': [
        'Sasha goes to Mannheim University',
        'Danylo will create a great application',
        'Danylo will not create a great application',
        'Dasha didnt fulfill her task'
    ],
    'class': [1, 1, 0, 0]
})

#### Preprocessing

In [290]:
from sklearn.pipeline import Pipeline
from notebooks.Dzim.web_mining.bert_processor import TextCleaner, SeriesConverter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from notebooks.Dzim.web_mining.feature_generation import FeatureGenerator

In [292]:
target = 'label'

In [293]:
# Split the data into train and test sets
data_train, data_test, y_train, y_test = train_test_split(data[['tweet']], data[target], test_size=0.2, random_state=42)

In [295]:
data_train.shape, data_test.shape

((25569, 1), (6393, 1))

In [297]:
# Define the pipeline
nlp = Pipeline([
    ("cleaning", TextCleaner()),
    ("series_converter", SeriesConverter()),
    ("vectorization", TfidfVectorizer(max_df=0.7, min_df=3)),
])

In [298]:
# Fit-transform the training data
X_train = nlp.fit_transform(data_train)  #.toarray()

In [299]:
# Transform the test data
X_test = nlp.transform(data_test)

In [155]:
# fg = FeatureGenerator()

In [156]:
# from sklearn.compose import ColumnTransformer
#
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("nlp", nlp, ['tweet']),
#         ("feat_gen", FeatureGenerator(), ['tweet']),
#     ],
#     remainder='drop'
# )

In [340]:
# fg.transform(data_sample)

#### Features

#### Models

In [301]:
from sklearn.dummy import DummyClassifier

dummy_mean = make_pipeline(
    DummyClassifier(strategy='most_frequent'),
)
dummy_median = make_pipeline(
    DummyClassifier(strategy='stratified'),
)

In [302]:
from sklearn.linear_model import LogisticRegression

lr = make_pipeline(
    # SimpleImputer(strategy='median', fill_value=0),
    LogisticRegression(),
)

In [303]:
from catboost import CatBoostClassifier

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

from sklearn.ensemble import RandomForestClassifier

ctb = make_pipeline(
    CatBoostClassifier()
)
xgb = make_pipeline(
    XGBClassifier()
)
lgbm = make_pipeline(
    LGBMClassifier()
)

rfc_parameters = {
    'max_depth': None,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'n_estimators': 100
}
rfc = make_pipeline(
    # SimpleImputer(missing_values=np.nan, strategy='mean'),
    RandomForestClassifier(**rfc_parameters),
)

In [310]:
model = rfc

In [305]:
lr.fit(X_train, y_train)

Pipeline(steps=[('logisticregression', LogisticRegression())])

In [311]:
model.fit(X_train, y_train)

Pipeline(steps=[('randomforestclassifier', RandomForestClassifier())])

In [312]:
prediction = model.predict(X_test)

In [313]:
import warnings
from sklearn.metrics import classification_report, confusion_matrix

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(classification_report(prediction, y_test))
    print(confusion_matrix(prediction, y_test))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      6118
           1       0.52      0.86      0.65       275

    accuracy                           0.96      6393
   macro avg       0.76      0.91      0.81      6393
weighted avg       0.97      0.96      0.96      6393

[[5899  219]
 [  38  237]]


In [314]:
train_prediction = model.predict(X_train)

In [315]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(classification_report(train_prediction, y_train))
    print(confusion_matrix(train_prediction, y_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23792
           1       0.99      1.00      1.00      1777

    accuracy                           1.00     25569
   macro avg       1.00      1.00      1.00     25569
weighted avg       1.00      1.00      1.00     25569

[[23782    10]
 [    1  1776]]


In [316]:
pipelines = [
    ("Dummy_mean", dummy_mean),
    ("Dummy_median", dummy_median),
    ("LinReg", lr),
    ("XGB", xgb),
    ("LGBM", lgbm),
    ("RFC", rfc),
    ("CatBoost", ctb),
]

#### Models

In [319]:
#### training
model_names, model_pipelines = zip(*pipelines)
metrics, weights = estimate_multiple_models(
    model_pipelines,
    lambda pipeline, _: estimate_model(
        pipeline,
        (X_train, X_test, y_train, y_test),
    ),
    model_names
)

C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dzmit\anaconda3\envs\eurostars\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Learning rate set to 0.041118
0:	learn: 0.6476334	total: 233ms	remaining: 3m 52s
1:	learn: 0.6072432	total: 321ms	remaining: 2m 40s
2:	learn: 0.5712074	total: 404ms	remaining: 2m 14s
3:	learn: 0.5370233	total: 489ms	remaining: 2m 1s
4:	learn: 0.5076169	total: 577ms	remaining: 1m 54s
5:	learn: 0.4808841	total: 660ms	remaining: 1m 49s
6:	learn: 0.4559781	total: 746ms	remaining: 1m 45s
7:	learn: 0.4341940	total: 842ms	remaining: 1m 44s
8:	learn: 0.4134218	total: 931ms	remaining: 1m 42s
9:	learn: 0.3941060	total: 1.02s	remaining: 1m 40s
10:	learn: 0.3767878	total: 1.12s	remaining: 1m 40s
11:	learn: 0.3614723	total: 1.21s	remaining: 1m 39s
12:	learn: 0.3479418	total: 1.3s	remaining: 1m 38s
13:	learn: 0.3353971	total: 1.4s	remaining: 1m 38s
14:	learn: 0.3246373	total: 1.49s	remaining: 1m 37s
15:	learn: 0.3136440	total: 1.57s	remaining: 1m 36s
16:	learn: 0.3047822	total: 1.68s	remaining: 1m 37s
17:	learn: 0.2966454	total: 1.77s	remaining: 1m 36s
18:	learn: 0.2894830	total: 1.87s	remaining: 1m

In [320]:
metrics

set             train                                                    test  \
metric       f1_score cohen_kappa_score precision_score recall_score f1_score   
model                                                                           
Dummy_mean      0.000             0.000           0.000        0.000    0.000   
Dummy_median    0.082             0.013           0.082        0.082    0.073   
LinReg          0.550             0.531           0.952        0.387    0.508   
XGB             0.670             0.653           0.956        0.515    0.558   
LGBM            0.662             0.644           0.941        0.510    0.560   
RFC             0.997             0.996           0.999        0.994    0.641   
CatBoost        0.680             0.663           0.958        0.527    0.589   

set                                                          
metric       cohen_kappa_score precision_score recall_score  
model                                                        
Dummy_mean               0.000           0.000        0.000  
Dummy_median             0.000           0.071        0.075  
LinReg                   0.487           0.904        0.353  
XGB                      0.536           0.838        0.419  
LGBM                     0.536           0.775        0.439  
RFC                      0.622           0.888        0.502  
CatBoost                 0.568           0.861        0.447

In [321]:
print(weights)

                  weight
model    feature        
LinReg   x155      6.861
         x6788     6.015
         x4997     5.476
         x4998     5.027
         x6853     4.392
...                  ...
CatBoost 4532      0.000
         4533      0.000
         4534      0.000
         4535      0.000
         0         0.000

[27969 rows x 1 columns]


In [322]:
for name, _ in pipelines:
    try:
        print("#" * 80)
        print(name)
        print(weights.loc[name])
    except KeyError:
        print('Not Supported')

################################################################################
Dummy_mean
Not Supported
################################################################################
Dummy_median
Not Supported
################################################################################
LinReg
         weight
feature        
x155      6.861
x6788     6.015
x4997     5.476
x4998     5.027
x6853     4.392
...         ...
x4445    -2.665
x627     -3.027
x2765    -3.027
x3679    -3.296
x1523    -4.132

[6993 rows x 1 columns]
################################################################################
XGB
Not Supported
################################################################################
LGBM
             weight
feature            
Column_6788   0.056
Column_155    0.053
Column_6382   0.051
Column_4998   0.039
Column_4997   0.030
...             ...
Column_4579   0.000
Column_4580   0.000
Column_4581   0.000
Column_4582   0.000
Column_0      0.000

[6992 rows x 1 colu

In [323]:
final_pipeline = Pipeline([
    ('NLP', nlp),
    ('regressor', rfc)
])

In [339]:
final_pipeline.predict_proba(pd.DataFrame({
    'tweet': [
        'Sasha goes to Mannheim University',
        '@Danylo will create a great application',
        '@You bitch suck a dick fuck you shit in your ass',
        'He will break it',
    ]}))

array([[1.00000000e+00, 0.00000000e+00],
       [9.40000000e-01, 6.00000000e-02],
       [7.50000000e-01, 2.50000000e-01],
       [9.99088757e-01, 9.11243187e-04]])

In [82]:
from src.common.prediction_model.persistence import FsModelPersistence
from src.common.validation.metrics import aggregate_metrics
from src.common.prediction_model.prediction_model import ModelContainer, ModelMetadata

container = ModelContainer.create(
    pipeline_name='bus_occupancy_prediction',
    pipeline=rfc,
    feature_names=[],
    metadata=ModelMetadata(
        model_name=str(lgbm.__class__.__name__),
        **aggregate_metrics(metrics)['test'].loc["RFC"]
    )
)

In [83]:
# %%
persistor = FsModelPersistence()
name = persistor.save(container)

In [84]:
name

'bus_occupancy_prediction_Pipeline_2023-04-17.bin.gz'

#### Hyperparameter optimization

In [343]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [20, 50, 100, 200],
    'max_depth': [None, 2, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create a RandomForestClassifier object
model = RandomForestClassifier()

"""
Best parameters:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best accuracy:  0.9592866361609763
Computation tmie: 245.936190366745
"""

In [347]:
# Define the parameter grid to search over
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],
    'random_strength': [0.1, 0.5, 1],
}
model = CatBoostClassifier()
"""

"""

'\n\n'

In [349]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'min_child_weight': [1, 3, 5],
}

model = XGBClassifier(tree_method='gpu_hist')

In [ ]:
# Create a GridSearchCV object and fit the data
start = time.time()
grid_search = GridSearchCV(model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and accuracy score
print("Best parameters: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)
end = time.time()
print(f"Computation tmie: {end - start}")